## Librerías

In [ ]:
import pandas as pd
from shapely import wkt
from shapely.ops import transform
import pyproj
from functools import partial
import folium

# Limpieza parques y jardines

## Filtrar por tamaño de "parcela"

###### No ejecutar mas

In [ ]:
# Cargar el archivo CSV
df = pd.read_csv('parques_y_jardines.csv')

# Convertir las cadenas de la columna 'geometry' a objetos geométricos usando Shapely
df['geometry'] = df['geometry'].apply(wkt.loads)

# Proyectar las geometrías para calcular el área en metros cuadrados
# Definir una función para cambiar la proyección a una que mida correctamente las áreas
def project_geom(geom):
    project = partial(
        pyproj.transform,
        pyproj.Proj(init='epsg:4326'),  # Coordenadas geográficas, WGS84
        pyproj.Proj(init='epsg:3857'))  # Sistema métrico, utilizado para medir áreas
    return transform(project, geom)

# Aplicar la proyección y calcular el área
df['area'] = df['geometry'].apply(lambda geom: project_geom(geom).area)

# Filtrar las geometrías por área, excluyendo aquellas menores a 100 m²
umbral_area = 100  # 100 metros cuadrados
df_filtrado = df[df['area'] > umbral_area]

# Guardar los resultados filtrados en un nuevo CSV
df_filtrado.to_csv('parques_y_jardines.csv', index=False)

# Retornar el número de filas del DataFrame filtrado para verificar
df_filtrado.shape[0]

### Mapear las parcelas

###### No ejecutar mas

In [ ]:
# Cargar el archivo CSV filtrado
df_filtrado = pd.read_csv('parques_y_jardines.csv')

# Convertir las cadenas de la columna 'geometry' a objetos geométricos usando Shapely
df_filtrado['geometry'] = df_filtrado['geometry'].apply(wkt.loads)

# Añadir una columna con etiquetas numéricas
if 'etiqueta' not in df_filtrado.columns:
    df_filtrado['etiqueta'] = range(1, len(df_filtrado) + 1)

# Crear un mapa de Folium
mapa = folium.Map(location=[42.3439, -3.6969], zoom_start=13)

# Añadir cada parque o jardín al mapa con relleno
for parque, row in df_filtrado.iterrows():
    popup_text = f"ID: {row['etiqueta']} - {row.get('name', 'Sin nombre')} - Área: {row['area']} m²"
    if row['geometry'].geom_type == 'Point':
        folium.Marker(
            location=[row['geometry'].y, row['geometry'].x],
            popup=popup_text
        ).add_to(mapa)
        
    elif row['geometry'].geom_type == 'Polygon':
        folium.Polygon(
            locations=[(point[1], point[0]) for point in row['geometry'].exterior.coords],
            popup=popup_text,
            color='#3186cc',  
            fill=True,
            fill_color='#3186cc', 
            fill_opacity=0.4  
        ).add_to(mapa)

# Guardar el mapa en un archivo HTML
mapa.save('mapa_parques_y_jardines.html')

# Guardar el DataFrame con la columna 'etiqueta' en un nuevo archivo CSV
df_filtrado.to_csv('parques_y_jardines.csv', index=False)

### Limpieza visual entre el mapa de folium y los datos del csv

#### Repetir cuantas veces necesitemos

In [ ]:
# Cargar el archivo CSV filtrado
df_filtrado = pd.read_csv('parques_y_jardines.csv')

In [ ]:
# Rangos a eliminar
rangos_a_eliminar = [(115, 126), (578, 630), (633, 662), (664, 777), (781, 867), 
                    (869 , 903), (908, 980), 
                    ]
# Números especificos a eliminar
numeros_especificos = [2790, 2789, 2791 ]

In [ ]:
# Convertir los rangos en una lista de números
numeros_a_eliminar = []
for rango in rangos_a_eliminar:
    numeros_a_eliminar.extend(range(rango[0], rango[1] + 1))

# Añadir los números específicos a la lista
numeros_a_eliminar.extend(numeros_especificos)

# Filtrar el DataFrame para excluir los números en la lista
df_filtrado = df_filtrado[~df_filtrado['etiqueta'].isin(numeros_a_eliminar)]

# Guardar los resultados en un nuevo archivo CSV
df_filtrado.to_csv('parques_y_jardines_filtrado.csv', index=False)


#### Mapear el resultado final

In [ ]:
# Cargar el DataFrame actualizado
df_filtrado_actualizado = pd.read_csv('parques_y_jardines_filtrado.csv')

# Convertir las cadenas de la columna 'geometry' a objetos geométricos usando Shapely
df_filtrado_actualizado['geometry'] = df_filtrado_actualizado['geometry'].apply(wkt.loads)

# Crear un nuevo mapa de Folium
mapa_actualizado = folium.Map(location=[42.3439, -3.6969], zoom_start=13)

# Añadir cada parque o jardín del DataFrame actualizado al mapa
for _, row in df_filtrado_actualizado.iterrows():
    popup_text = f"ID: {row['etiqueta']} - {row.get('name', 'Sin nombre')} - Área: {row['area']} m²"
    if row['geometry'].geom_type == 'Point':
        folium.Marker(
            location=[row['geometry'].y, row['geometry'].x],
            popup=popup_text
        ).add_to(mapa_actualizado)
    elif row['geometry'].geom_type == 'Polygon':
        folium.Polygon(
            locations=[(point[1], point[0]) for point in row['geometry'].exterior.coords],
            popup=popup_text,
            color='#3186cc',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7
        ).add_to(mapa_actualizado)

# Guardar el nuevo mapa en un archivo HTML
mapa_actualizado.save('mapa_parques_y_jardines_actualizado.html')


##### Repetimos limpieza